In [10]:
import textract
from words import not_so_naive_split, reduce_dictionary, sentence_vector
import numpy as np
import json
from gensim.models import Doc2Vec

from keras.models import model_from_json
from keras.preprocessing import sequence

from pretty_testing import predict_test

In [20]:
pdf_names = ['T3LAB-INJENIA_Analisi Preliminare.pdf','../atti_costitutivi/5122462300001.pdf',
             '../atti_costitutivi/5122464750001.pdf']
#labels = ['...']

In [9]:
gensim_model = Doc2Vec.load('gensim_model.d2v')

with open('keras_model.json') as f:
    keras_model = model_from_json(f.read())
keras_model.load_weights("keras_weights.h5")

with open('reduced_dictionary.json') as f:
    permitted_words = set(json.load(f))

In [24]:
def embed_document(model, doc, permitted_words):
    return [sentence_vector(model, sentence, permitted_words) for sentence in doc]

def predict_documents(filenames, gensim_model, keras_model, permitted_words):
    try:
        txts = [textract.process(filename) for filename in filenames]
    except:
        print "C'è un documento illeggibile..." 
        return
        
    splitted_txts = [not_so_naive_split(txt) for txt in txts]
    
    filtered_txts = [list(reduce_dictionary(document, permitted_words)) for document in splitted_txts]
    
    embedded_txts = [embed_document(gensim_model, doc, permitted_words) for doc in filtered_txts]
    
    padded_data = sequence.pad_sequences(embedded_txts, maxlen=200, padding="pre", truncating="post", value=0.0, dtype='float32')
    
    labels_map = ['non costitutivo', 'costitutivo']
    print [labels_map[pred[0]] for pred in keras_model.predict_classes(padded_data, verbose=0)]

In [23]:
predict_documents(pdf_names, gensim_model, keras_model, permitted_words)

Documento illeggibile
